In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import time
import math

In [2]:
# Adjust the model to get a higher performance
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 首先找到Net的父类（比如是类nn.Module），然后把类Net的对象self转换为类nn.Module的对象，然后“被转换”的类nn.Module对象调用自己的init函数
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # 四个参数分别是in_channels, out_channels,kernel_size,stride = 1
        # in_channels:就是输入的四维张量[N, C, H, W]中的C了，即输入张量的channels数。这个形参是确定权重等可学习参数的shape所必需的。
        # out_channels:即期望的四维输出张量的channels数，不再多说。
        # out_channels:即期望的四维输出张量的channels数，不再多说。
        # stride = 1:卷积核在图像窗口上每次平移的间隔，即所谓的步长。这个概念和Tensorflow等其他框架没什么区别，不再多言。
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x


In [3]:
# 训练模型
# trget是真是的label
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    plt.figure()
    pic = None
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx in (1,2,3,4,5):
            if batch_idx == 1:
                pic = data[0,0,:,:]
            else:
                pic = torch.cat((pic,data[0,0,:,:]),dim=1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        #print("------target is--------"+str(target))
        #print("------the actual output is--------"+str(output))
        # Calculate gradients
        loss.backward()
        # Optimize the parameters according to the calculated gradients
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break
    #print(data.byte().type())
    #plt.imshow(data, cmap='gray')
    #plt.show()

In [4]:
def as_num(x):
    y='{:.7f}'.format(x)
    return(y)

In [5]:
# 测试模型
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            '''
            print("####the output is#####")
            #print(output)
            for i in range(0,1000):
                print("jjjjjjjjjj")
                #print(tem[i,:])
                z_exp = [math.exp(n) for n in output[i,:]]
                sum_z_exp = sum(z_exp)
                softmax = [n/sum_z_exp for n in z_exp]
                print(z_exp)
                print(sum_z_exp)
                print('------------------')
                final = [float(as_num(n)) for n in softmax]
                rate1 = round(final[0], 6)
                percent = [format(n, '.4%') for n in final]
                print(percent)
            '''
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print("####taget#####  "+str(target))
            #print("#########  "+str(target.size()))
            #print("####the output is#####  "+str(pred))
            #print("#########  "+ str(pred.size()))

    test_loss /= len(test_loader.dataset)
    
    # 保留五位小数点
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.5f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# 测试模型
record = []
def test_draw(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #print("####the output is#####  "+str(output))
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print("####taget#####  "+str(target))
            print("####the output is#####  "+str(pred))
            for i in range(0,len(target)):
                if int('%i'%target[i]) != int('%i'%pred[i][0]):
                  record.append(i)
                  #print('the the the i i i i i  is is is'+str(i))
                  print('######!!!!!!!!!target is: '+ str('%i'%target[i]))
                  print('######prediction is: ' + str('%i'%pred[i][0]))

    #test_loss /= len(test_loader.dataset)
    
    # 保留五位小数点
    print(len(record))
    #print(record)
    #print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.5f}%)\n'.format(
        #test_loss, correct, len(test_loader.dataset),
        #100. * correct / len(test_loader.dataset)))
    fig = plt.figure()
    for i in range(0,15):
        plt.subplot(3,5,i+1)
        plt.tight_layout()
        j = record[i]
        #print('wbcgbdWHCOWAJCWCWBCWAHC.OWCF,WBDVCBWDCBW.CD')
        #print('the j is '+ str(j))
        #print("----------------taget is   "+str(target[j]))
        #print("Prediction is   "+str('%i'%pred[j][0]))
        plt.imshow(data[j][0], cmap='gray', interpolation='none')
        plt.title('label: %i'% target[j]+'\nPrediction: %i' %pred[j+2][0],fontsize = 9)
        plt.subplots_adjust(left=None, bottom=None, right=0.9, top=None, wspace=None, hspace=1.2)
    fig

In [7]:
def main():
    # Training settings
    
    # argparse是一个Python模块,是一个用来解析命令行程序的参数的模块：命令行选项、参数和子命令解析器
    # argparse模块可以让人轻松编写用户友好的命令行接口。程序定义它需要的参数。然后argparser将弄清
    # 1.创建对象
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    # 2.添加参数
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=14, metavar='N',
                        help='number of epochs to train (default: 14)')
    parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                        help='learning rate (default: 1.0)')
    parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                        help='Learning rate step gamma (default: 0.7)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--dry-run', action='store_true', default=False,
                        help='quickly check a single pass')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    # 3.解析参数
    args = parser.parse_args(args=[])
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    # batch_size is a crucial hyper-parameter
    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    
    
    if use_cuda:
        # Adjust num worker and pin memory according to your computer performance
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    # Normalize the input (black and white image)
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

    # Make train dataset split
    dataset1 = datasets.MNIST('./data', train=True, download=True,
                       transform=transform)
    # Make test dataset split
    dataset2 = datasets.MNIST('./data', train=False,
                       transform=transform)

    # Convert the dataset to dataloader, including train_kwargs and test_kwargs
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    # Put the model on the GPU or CPU
    model = Net().to(device)

    # Create optimizer
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    # Create a schedule for the optimizer
    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

    # Begin training and testing
    '''
    count = 0
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        count +=1
        if count == 1:
            test_draw(model, device, test_loader)
            break
        scheduler.step()
        
    ''' 
    count = 0
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        count +=1
        scheduler.step()

    # Save the model
    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.pt")

In [8]:
start = time.perf_counter()

main()

end = time.perf_counter()
print('Running time: %s Seconds' % (end - start))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305400
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.359781
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.830669
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.605963
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.345928
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.449979
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.293631
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.282048
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.549668
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.214355
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.265443
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.319265
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.207352
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.241294
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.284134
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.112255
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.260232
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.107072
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.449265
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.257380
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.060786
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.060687
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.107201
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.065608
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.056637
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.070513
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.023972
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.019916
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.032087
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.069121
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.125663
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.039719
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.005861
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.066138
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.101269
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.041555
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.030685
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.097013
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.021265
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.035221


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.003562
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.012118
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.008337
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.003541
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.004653
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.121320
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.003857
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.030477
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.007013
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.009024
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.035746
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.051549
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.001367
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.025495
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.160169
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.111738
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.004519
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.030480
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.009963
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.046312


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.046592
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.010936
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.006134
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.054821
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.016677
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.037660
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.058409
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.003131
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.015880
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.009030
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.050336
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.035648
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.001791
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.022056
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.001864
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.057736
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.081103
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.030762
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.001183
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.003635


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.068734
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.007073
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.000566
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.000181

Test set: Average loss: 0.0264, Accuracy: 9912/10000 (99.12000%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.003679
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.002346
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.034686
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.042950
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.001419
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.014012
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.003759
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.001757
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.114214
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.064554
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.122055
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.173440
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.021021
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.016701
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.001636
Train Epoch

...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [9]:
test_kwargs = 1000
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

dataset2 = datasets.MNIST('./data', train=False,
                       transform=transform)
test_loader = torch.utils.data.DataLoader(dataset2, test_kwargs)
print(test_loader.dataset)

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [10]:
test_loader

In [ ]:
test_loader.dataset.type()

In [ ]:
datasets.MNIST

In [ ]:
#import pandas as pd #lets us handle data as dataframes
#data = pd.read_csv('train.csv')

In [ ]:
'''
image = data.iloc[:,1:].values
images = omages.astype(np.float)

# convert from [0:255] => [0.0:1.0]
images = np.multiply(images, 1.0/255.0)

print('images({0[0]},{0[1]})'.format(images.shape))
'''